In [7]:
!sudo apt update
!sudo apt -y install build-essential vim
!pip3 install networkx node2vec python-Levenshtein nodevectors emblaze pymde

import pandas as pd
import json
import networkx as nx
from node2vec import Node2Vec
from gensim.models import KeyedVectors
import numpy as np
from collections import defaultdict

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
14 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
vim is already the newest version (2:8.1.2269-1ubuntu5.7).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [3]:
all_animes_df = pd.read_csv('./work/data/processed-metadata.csv')

all_animes = []
anime_title_by_id = {}
anime_id_by_title = {}
anime_ix_by_id = {}
anime_by_id = {}

i = 0
for anime in all_animes_df.itertuples(index=False):
    all_animes.append(anime)
    anime_title_by_id[anime.id] = anime.title
    anime_id_by_title[anime.title] = anime.id
    anime_ix_by_id[anime.id] = i
    anime_by_id[anime.id] = anime
    i += 1

In [70]:
all_ratings = pd.read_csv('./work/data/collected_animelists.csv')

In [74]:
# only retain the "username", "anime_id", and "my_score" columns
ratings = all_ratings[['username', 'anime_id', 'my_score']]
# only retain rows where "my_score" is not null and greater than or equal to 6
ratings = ratings[ratings['my_score'].notnull()]
ratings = ratings[ratings['my_score'] > 0]
ratings = ratings[ratings['anime_id'].isin(anime_title_by_id.keys())]
ratings['anime_ix'] = ratings['anime_id'].apply(lambda x: anime_ix_by_id[x])

def scale_rating(rating: int) -> float:
    if rating == 10:
        return 10.1
    if rating == 9:
        return 9.1
    if rating == 8:
        return 6.5
    if rating == 7:
        return 3.3
    if rating == 6:
        return 0.5
    if rating == 5:
        return -1.5
    if rating == 4:
        return -4.5
    if rating == 3:
        return -7.5
    if rating == 2:
        return -10.5
    if rating == 1:
        return -13.5
    raise ValueError("Invalid rating: {}".format(rating))

# scale ratings from score to our custom scale
ratings['my_score'] = ratings['my_score'].apply(scale_rating)
ratings = ratings[ratings['my_score'] > 0]

print("Remaining ratings:", ratings.shape[0])
print(ratings.head())

# collect python garbage
import gc
gc.collect()

Remaining ratings: 66596570
   username  anime_id  my_score  anime_ix
2     Jakly     28339       0.5      6181
7     Jakly     13971       0.5      4903
8     Jakly     32998       3.3      7093
9     Jakly     33964       3.3      7371
10    Jakly      6094       3.3      3310


74

In [131]:
# ratings_subset = ratings.sample(n=5000000)
# ratings_subset = ratings[:5000000]
ratings_subset = ratings

# Retain only ratings of anime that have >=100 ratings
ratings_subset = ratings_subset[ratings['anime_id'].isin(anime_title_by_id.keys())]
print(f"Retained {ratings_subset.shape[0]} ratings out of {ratings.shape[0]} total")

Retained 66596570 ratings out of 66596570 total


In [132]:
from collections import defaultdict

ratings_by_username = defaultdict(list)

i = 0
for row in ratings_subset.itertuples():
    if i % 1000000 == 0:
        print(f"Processed {i} rows")
    i += 1

    rating = row.my_score
    if rating < 0:
        continue
    anime_ix = anime_ix_by_id[row.anime_id]
    ratings_by_username[row.username].append((anime_ix, rating))

Processed 0 rows
Processed 1000000 rows
Processed 2000000 rows
Processed 3000000 rows
Processed 4000000 rows
Processed 5000000 rows
Processed 6000000 rows
Processed 7000000 rows
Processed 8000000 rows
Processed 9000000 rows
Processed 10000000 rows
Processed 11000000 rows
Processed 12000000 rows
Processed 13000000 rows
Processed 14000000 rows
Processed 15000000 rows
Processed 16000000 rows
Processed 17000000 rows
Processed 18000000 rows
Processed 19000000 rows
Processed 20000000 rows
Processed 21000000 rows
Processed 22000000 rows
Processed 23000000 rows
Processed 24000000 rows
Processed 25000000 rows
Processed 26000000 rows
Processed 27000000 rows
Processed 28000000 rows
Processed 29000000 rows
Processed 30000000 rows
Processed 31000000 rows
Processed 32000000 rows
Processed 33000000 rows
Processed 34000000 rows
Processed 35000000 rows
Processed 36000000 rows
Processed 37000000 rows
Processed 38000000 rows
Processed 39000000 rows
Processed 40000000 rows
Processed 41000000 rows
Processe

In [133]:
import numpy as np

cooccurrence_matrix = np.zeros((len(anime_title_by_id), len(anime_title_by_id)))

i = 0
print(f"Users to process: {len(ratings_by_username)}")

for username, ratings_for_user in ratings_by_username.items():
    i += 1
    if i % 8000 == 0:
        print(f"Processed {i} users")

    for anime_ix_0, rating_0 in ratings_for_user:
        for anime_ix_1, rating_1 in ratings_for_user:
            if anime_ix_0 == anime_ix_1:
                continue

            cooccurrence_matrix[anime_ix_0, anime_ix_1] += rating_0 * rating_1

Users to process: 409923
Processed 8000 users
Processed 16000 users
Processed 24000 users
Processed 32000 users
Processed 40000 users
Processed 48000 users
Processed 56000 users
Processed 64000 users
Processed 72000 users
Processed 80000 users
Processed 88000 users
Processed 96000 users
Processed 104000 users
Processed 112000 users
Processed 120000 users
Processed 128000 users
Processed 136000 users
Processed 144000 users
Processed 152000 users
Processed 160000 users
Processed 168000 users
Processed 176000 users
Processed 184000 users
Processed 192000 users
Processed 200000 users
Processed 208000 users
Processed 216000 users
Processed 224000 users
Processed 232000 users
Processed 240000 users
Processed 248000 users
Processed 256000 users
Processed 264000 users
Processed 272000 users
Processed 280000 users
Processed 288000 users
Processed 296000 users
Processed 304000 users
Processed 312000 users
Processed 320000 users
Processed 328000 users
Processed 336000 users
Processed 344000 users

In [135]:
# Save the co-occurrence matrix to compressed binary numpy file
np.savez_compressed('./work/data/cooccurrence_matrix.npz', cooccurrence_matrix)

In [5]:
cooccurrence_matrix = np.load('./work/data/cooccurrence_matrix.npz')['arr_0']

In [8]:
import json
import math

def score_num_recommendations(num_recommendations):
    return math.sqrt(num_recommendations) / 2.

def parse_extra(item):
    related = json.loads(item.related_anime) 
    related = [(r['node']['id'], 0.2 if r['relation_type'] == 'character' else 40) for r in related]
    recommended = [(r['node']['id'], score_num_recommendations(r['num_recommendations'])) for r in json.loads(item.recommendations)]
    return [(anime_ix_by_id.get(id), score) for id, score in related + recommended if anime_ix_by_id.get(id) is not None]

related_by_anime_ix = defaultdict(lambda: defaultdict(float))

for item in all_animes:
    anime_ix = anime_ix_by_id.get(item.id)
    if anime_ix is None:
        continue
    related_for_item = parse_extra(item)
    entry = related_by_anime_ix[anime_ix]
    for ix, weight in related_for_item:
        entry[ix] += weight
        
kill_me_baby_id = 11079
kill_me_baby_ix = anime_ix_by_id[kill_me_baby_id]
[(score, all_animes[ix].title) for ix, score in related_by_anime_ix[kill_me_baby_ix].items()]

[(40.0, 'Kill Me Baby: Butsuzou Kegatte Nise Halloween'),
 (1.224744871391589, 'Lucky☆Star'),
 (1.118033988749895, 'Nichijou'),
 (1.118033988749895, 'Acchi Kocchi'),
 (1.0, 'Yuyushiki'),
 (0.8660254037844386, 'Danshi Koukousei no Nichijou'),
 (0.8660254037844386, 'Aho Girl'),
 (0.8660254037844386, 'Akuma no Riddle'),
 (0.7071067811865476, 'Chitose Get You!!'),
 (0.7071067811865476, 'Hetalia Axis Powers'),
 (0.7071067811865476, 'Gugure! Kokkuri-san')]

In [9]:
cooccurrence_matrix_wextra = cooccurrence_matrix.copy()

for anime_0_ix in range(len(cooccurrence_matrix)):
    related_for_anime0 = related_by_anime_ix[anime_0_ix]
    max_weight = cooccurrence_matrix[anime_0_ix].max()

    for ix, weight in related_for_anime0.items():
        if ix == anime_0_ix:
            continue
        cooccurrence_matrix_wextra[anime_0_ix, ix] += weight * max_weight

np.save('./work/data/cooccurrence_matrix_wextra.npy', cooccurrence_matrix_wextra)

In [138]:
graph = nx.Graph()

def get_topn(weights, topn):
    return sorted(enumerate(weights), key=lambda x: x[1], reverse=True)[:topn]

topn = 40

def get_extra_topn(rating_count):
    return int(max(math.pow(rating_count, 0.46) * 0.4 - 10, 0))

for anime_0_ix in range(cooccurrence_matrix.shape[0]):
    row = cooccurrence_matrix[anime_0_ix]
    top_edges = get_topn(row, topn)
    top_related_anime_ixs_set = set([ix for ix, weight in top_edges])

    top_magnitude = 0 if len(top_edges) == 0 else top_edges[0][1]
    if top_magnitude == 0:
        continue
    related_for_anime0 = related_by_anime_ix[anime_0_ix]

    for related_anime_ix, related_weight in related_for_anime0.items():
        if related_anime_ix in top_related_anime_ixs_set:
            continue
        top_edges.append((related_anime_ix, 0.))

    for anime_1_ix, base_weight in top_edges:
        extra_weight = related_for_anime0[anime_1_ix]
        # This may need to be tuned
        weight = base_weight + extra_weight * top_magnitude * 1.
        graph.add_edge(anime_0_ix, anime_1_ix, weight=weight)

print("Edges:", graph.number_of_edges())

Edges: 423247


In [139]:
import csrgraph as cg
import nodevectors

cgraph = cg.csrgraph(graph)

/opt/conda/lib/python3.9/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


In [140]:
dimensions = 10
order = 2
algo = "ggvec"

embedding_model = None
if algo == "ggvec":
    embedding_model = nodevectors.GGVec(n_components=dimensions, learning_rate=0.01, negative_ratio=0.6, verbose=True, order=order)
elif algo == "node2vec":
    embedding_model = nodevectors.Node2Vec(walklen=8, epochs=50, return_weight=2., neighbor_weight=1., n_components=dimensions, threads=14)
elif algo == "ProNE":
    embedding_model = nodevectors.ProNE(n_components=dimensions)

embeddings = embedding_model.fit_transform(cgraph)

Loss: 0.0036	:  13%|█▎        | 46/350 [01:41<11:08,  2.20s/it]

Converged! Loss: 0.0036


In [142]:
keys = [int(n) for n in cgraph.nodes()]

print(f"key length: {len(keys)}; weight length: {len(embeddings)}; anime_count: {len(all_animes)}")

fname = f"./work/data/embedding_{algo}_full_posonly_wextra_top{topn}_{dimensions}d_order{order}.w2v"
with open(fname, 'wt') as f:
    tab = ' '
    nl = '\n'
    f.write(f"{len(keys)}{tab}{dimensions}{nl}")
    for key, embedding in zip(keys, embeddings):
        f.write(f"{key}{tab}{tab.join(map(str, embedding))}{nl}")
fname

key length: 10097; weight length: 10097; anime_count: 10097


'./work/data/embedding_ggvec_full_posonly_wextra_top40_10d_order2.w2v'